In [51]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table

In [52]:
class Params(object):
    def __init__(self):
        pass
    
targets = ['TRAPPIST-1 {0}'.format(s) for s in list('bcdefgh')]

def quadratic_to_kipping(u):
    q1 = (u[0] + u[1])**2
    q2 = 0.5 * u[0] / (u[0] + u[1])
    return np.array([q1, q2])

In [53]:
from collections import OrderedDict

shapes = OrderedDict([['{0}_samples.txt'.format(s.lower().replace('-', '').replace(' ', '')), [8, 20, 10000]] 
                      for s in targets])

paths = OrderedDict([['{0}_samples.txt'.format(s.lower().replace('-', '').replace(' ', '')),  
                      'r0, r1, q1, q2, a, inc, f0, t0'.split(', ')] 
                      for s in targets])

In [54]:
from collections import OrderedDict

# shapes = OrderedDict([['tres2_samples.txt', [5, 14, 10000]],
#          ['hat7_samples_fix_ars_ld.txt', [9, 18, 5000]], 
#          ['hat11_samples_ld_nomask.txt', [8, 16, 10000]], 
#          ['kepler17_samples_ld.txt', [9, 18, 10000]], 
#          ['kepler39_samples_ld.txt', [9, 18, 10000]], 
#          ['kepler412_samples_ld.txt', [9, 18, 10000]], 
#          #['kepler435_samples.txt', 'r0, r1, b, t14, f0'.split(', ')], 
#          ['gj1214_samples.txt', [7, 14, 10000]], 
#          ['hd80606_samples.txt', [6, 30, 5000]], 
#          ['k2trappist1/trappist1b_samples.txt', [8, 20, 10000]],
#          ['k2trappist1/trappist1c_samples.txt', [8, 30, 20000]]])

# paths = OrderedDict([['tres2_samples.txt', 'r0, r1, b, t14, f0'.split(', ')],
#          ['hat7_samples_fix_ars_ld.txt', 'r0, r1, a, i, f0, c1, c2, c3, c4'.split(', ')], 
#          ['hat11_samples_ld_nomask.txt', 'r0, r1, a, i, f0, t0, q1, q2'.split(', ')], 
#          ['kepler17_samples_ld.txt', 'r0, r1, M_star, R_star, inc, f0, t0, q1, q2'.split(', ')], 
#          ['kepler39_samples_ld.txt', 'r0, r1, M_star, R_star, inc, f0, t0, q1, q2'.split(', ')], 
#          ['kepler412_samples_ld.txt', 'r0, r1, M_star, R_star, inc, f0, t0, q1, q2'.split(', ')], 
#          #['kepler435_samples.txt', 'r0, r1, b, t14, f0'.split(', ')], 
#          ['gj1214_samples.txt', 'r0, r1, q1, q2, b, t14, f0'.split(', ')], 
#          ['hd80606_samples.txt', 'r0, r1, b, t14, f0, t0'.split(', ')], 
#          ['k2trappist1/trappist1b_samples.txt', 'r0, r1, q1, q2, b, t14, f0, t0'.split(', ')],
#          ['k2trappist1/trappist1c_samples.txt', 'r0, r1, q1, q2, b, t14, f0, t0'.split(', ')]])

In [55]:
params = []
for path, shape in zip(paths, shapes): 
    print(path, shapes[shape])
    p = Params()
    flatchain = np.loadtxt(path)
    ndim, nwalkers, nsamples = shapes[shape]
    chains = flatchain.reshape((nwalkers, nsamples, ndim))
    samples_burnin = chains[:, -1000:, :].reshape((-1, ndim))
    
    for attr, col in zip(paths[path], samples_burnin.T): 
        setattr(p, attr, col)
    
    params.append(p)

trappist1b_samples.txt [8, 20, 10000]
trappist1c_samples.txt [8, 20, 10000]
trappist1d_samples.txt [8, 20, 10000]
trappist1e_samples.txt [8, 20, 10000]
trappist1f_samples.txt [8, 20, 10000]
trappist1g_samples.txt [8, 20, 10000]
trappist1h_samples.txt [8, 20, 10000]


In [56]:
attrs_list = 'r0, r1, eps, sign, CI'.split(', ')
# attrs_list = 'b'.split(', ')
rows = []
for target, param in zip(targets, params):
    row = []
    for attr in attrs_list:
        if hasattr(param, attr):
            l, m, u = np.percentile(getattr(param, attr), [16, 50, 84])
            row.append([m, m-l, u-m])
        elif attr == 'eps':
            p1p02 = (param.r1/param.r0)**2
            
            l, m, u = np.percentile(1 - p1p02, [16, 50, 84])
            row.append([m, m-l, u-m])

        elif attr == 'sign':
            if np.median(param.r1) > np.median(param.r0): 
                row.append('$p_0 < p_1$')
            else: 
                row.append('$p_0 > p_1$')
        elif attr == 'CI': 
            if np.median(param.r0) < np.median(param.r1): 
                #row.append('$p_0 < p_1$')
                row.append(np.count_nonzero(param.r0 < np.median(param.r1))/len(param.r0))
            else: 
                row.append(np.count_nonzero(param.r0 > np.median(param.r1))/len(param.r0))
                #row.append('$p_0 > p_1$')            
        else: 
            row.append(None)
    rows.append(row)

In [57]:

formatted_rows = []
for target, row in zip(targets, rows):
    formatted_row = []
    formatted_row.append(target)
    for parameter in row:
        if type(parameter) is list:
            #formatted_row.append("${{{0}}}_{{-{1}}}^{{+{2}}}$".format(*list(map(lambda x: to_precision(x, 2), parameter))))
            formatted_row.append("${{{0}}}_{{-{1}}}^{{+{2}}}$".format(*list(map(lambda x: "{0:.4g}".format(x), parameter))))
        elif type(parameter) is str: 
            formatted_row.append(parameter)
        elif isinstance(parameter, float): 
            formatted_row.append("{0:.2f}\%".format(100*parameter))
        else: 
            formatted_row.append("")
    formatted_rows.append(formatted_row)
print(formatted_rows)

[['TRAPPIST-1 b', '${0.04947}_{-0.01186}^{+0.01302}$', '${0.08493}_{-0.000492}^{+0.0004234}$', '${-1.942}_{-2.151}^{+1.085}$', '$p_0 < p_1$', '99.59\\%'], ['TRAPPIST-1 c', '${0.04417}_{-0.01807}^{+0.01412}$', '${0.0834}_{-0.0004485}^{+0.0003813}$', '${-2.562}_{-6.664}^{+1.508}$', '$p_0 < p_1$', '99.94\\%'], ['TRAPPIST-1 d', '${0.05905}_{-0.02818}^{+0.02442}$', '${0.06096}_{-0.0005848}^{+0.0005777}$', '${-0.06128}_{-2.843}^{+0.5222}$', '$p_0 < p_1$', '52.95\\%'], ['TRAPPIST-1 e', '${0.04067}_{-0.02555}^{+0.01593}$', '${0.06924}_{-0.0005269}^{+0.0004409}$', '${-1.891}_{-17.95}^{+1.388}$', '$p_0 < p_1$', '97.28\\%'], ['TRAPPIST-1 f', '${0.06651}_{-0.01404}^{+0.01369}$', '${0.07953}_{-0.0005425}^{+0.0005217}$', '${-0.4291}_{-0.866}^{+0.4405}$', '$p_0 < p_1$', '82.65\\%'], ['TRAPPIST-1 g', '${0.06707}_{-0.01326}^{+0.01241}$', '${0.08586}_{-0.0006008}^{+0.0005228}$', '${-0.6398}_{-0.8967}^{+0.4697}$', '$p_0 < p_1$', '93.45\\%'], ['TRAPPIST-1 h', '${0.02651}_{-0.01869}^{+0.02626}$', '${0.0575

In [58]:
len(['Target'] + attrs_list), len(formatted_rows[0])

(6, 6)

In [59]:
table = Table(rows=formatted_rows, names=['Target'] + attrs_list)

In [60]:
table

Target,r0,r1,eps,sign,CI
str12,str33,str37,str31,str11,str7
TRAPPIST-1 b,${0.04947}_{-0.01186}^{+0.01302}$,${0.08493}_{-0.000492}^{+0.0004234}$,${-1.942}_{-2.151}^{+1.085}$,$p_0 < p_1$,99.59\%
TRAPPIST-1 c,${0.04417}_{-0.01807}^{+0.01412}$,${0.0834}_{-0.0004485}^{+0.0003813}$,${-2.562}_{-6.664}^{+1.508}$,$p_0 < p_1$,99.94\%
TRAPPIST-1 d,${0.05905}_{-0.02818}^{+0.02442}$,${0.06096}_{-0.0005848}^{+0.0005777}$,${-0.06128}_{-2.843}^{+0.5222}$,$p_0 < p_1$,52.95\%
TRAPPIST-1 e,${0.04067}_{-0.02555}^{+0.01593}$,${0.06924}_{-0.0005269}^{+0.0004409}$,${-1.891}_{-17.95}^{+1.388}$,$p_0 < p_1$,97.28\%
TRAPPIST-1 f,${0.06651}_{-0.01404}^{+0.01369}$,${0.07953}_{-0.0005425}^{+0.0005217}$,${-0.4291}_{-0.866}^{+0.4405}$,$p_0 < p_1$,82.65\%
TRAPPIST-1 g,${0.06707}_{-0.01326}^{+0.01241}$,${0.08586}_{-0.0006008}^{+0.0005228}$,${-0.6398}_{-0.8967}^{+0.4697}$,$p_0 < p_1$,93.45\%
TRAPPIST-1 h,${0.02651}_{-0.01869}^{+0.02626}$,${0.05758}_{-0.0008382}^{+0.0008506}$,${-3.684}_{-49.59}^{+3.492}$,$p_0 < p_1$,88.05\%


In [61]:
[table.rename_column(old, new) for old, new in zip(table.colnames, ['Target', '$p_0$', '$p_1$', '$\epsilon$', 'Condition', "CI"])]

[None, None, None, None, None, None]

In [62]:
table['Target'] = [s[-1] for s in table['Target']]

In [63]:
ascii.write(table, format='aastex', output='/Users/bmmorris/Dropbox/Apps/ShareLaTeX/trappist1_robin/results.tex',
            latexdict={'col_align': 'lccccr', 'tabletype': 'deluxetable*'}, overwrite=True)